<a href="https://colab.research.google.com/github/obielin/Palestine_News_Analysis/blob/main/Text_Clustering_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
import warnings
plt.style.use('ggplot')
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
#Loading the dataset
file_path = "/content/drive/MyDrive/cleaned_news_data.csv"
df= pd.read_csv(file_path)
df.head(5)

,headline,description,date,headline_word_count,description_word_count,stop_word_count,url_count
0,Pho­tos: Fears over Al-Aqsa stained-glass win­...,israeli raids alaqsa mosque compound led windo...,2014-03-10,6,15,5,0
1,The War in June 1967,june 1967 arabisraeli war lasted six days cons...,2017-02-06,5,20,6,0
2,Pales­tine and the UN’s ‘re­spon­si­bil­i­ty t...,p2p doctrine un must deploy palestine protect ...,2021-01-06,8,18,5,0
3,Fear bar­ri­er bro­ken de­spite hun­dreds of a...,israeli forces attempting reassert control pal...,2021-01-06,9,17,6,0
4,Is­rael rolls back gen­er­al’s com­ments on bo...,benny gantz says military chief spoke figurati...,2021-01-06,10,15,2,0


In [ ]:
%pip install umap-learn
%pip install plotly
%pip install transformers
%pip install sentence-transformers

In [ ]:
from sklearn.mixture import GaussianMixture
from umap import UMAP
import plotly.express as px
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Step 1: Text Vectorization on the 'description' column
vectorizer = TfidfVectorizer()
tfidf_embeddings = vectorizer.fit_transform(df['description'])

# Step 2: Dimensionality Reduction with UMAP
umap_model = UMAP(n_components=3, random_state=42, n_neighbors=15, min_dist=0.3, metric='euclidean')
reduced_embeddings = umap_model.fit_transform(tfidf_embeddings)

In [ ]:
reduced_embeddings.shape

(3338, 3)

In [ ]:
import plotly.graph_objects as go

def create_3d_scatter_plot(embeddings, hover_text, title):
    # Use one of the dimensions for coloring
    color = embeddings[:, 0]

    fig = go.Figure(data=[go.Scatter3d(
        x=embeddings[:, 0],
        y=embeddings[:, 1],
        z=embeddings[:, 2],
        mode='markers',
        text=hover_text,
        marker=dict(
            size=5,
            opacity=0.8,
            color=color,  # Assign colors based on the first dimension
            colorscale='Viridis',  # Color scale
            colorbar=dict(title='Color Scale')  # Color scale bar
        )
    )])

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='UMAP Component 1',
            yaxis_title='UMAP Component 2',
            zaxis_title='UMAP Component 3'
        ),
        hovermode='closest'
    )

    return fig

In [ ]:
# Example usage
fig = create_3d_scatter_plot(reduced_embeddings, df['description'], '3D Visualization of UMAP Embeddings')
fig.show()